In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_original, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti SUPER. Max memory: 15.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


In [2]:
model = FastLanguageModel.get_peft_model(
    model_original,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

knowledge = "Hints: 1. Reply Length: Longer replies tend to be more persuasive, as they can convey more information and elaborate on points effectively. 2. Language Dissimilarity with Original Post: Persuasive replies use different content words but match in stopwords. 3. Links and Evidence: Including links as evidence in an argument increases the chances of persuasion. 4. Calmer Tone: Replies that use calmer, less intense language are more likely to persuade, as they come across as more composed. 5. Positive Emotion and Sentiment: Persuasive replies include a mix of positive and negative sentiment."
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

def formatting_prompts_with_knowledge(examples):
     examples["instruction"] = [ins + "\n" + knowledge for ins in examples["instruction"]]
     return formatting_prompts(examples)

from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": "pairs_train_alpaca.jsonl",
                                           "test": "pairs_test_alpaca.jsonl",})

dataset_train = dataset["train"]
dataset_train = dataset_train.map(formatting_prompts_with_knowledge, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
dataset_train[0]

{'input': 'Original Post:\nCMV: Anything that is man-made is natural.\nI can\'t remember the topic that spurred this discussion, but a friend and I were debating whether man-made things were natural. He took the position that they are unnatural. \n\nHe cited this definition by Merriam-Webster:  existing in nature and not made or caused by people : coming from nature (http://www.merriam-webster.com/dictionary/natural) as his basis for the distinction for natural vs. unnatural.\n\nHowever, I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankind\'s capacity to create, problem-solve, analyze, rationalize, and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability, why is it that we would give the credit of these things solely to man, as opposed to nature? We are not separate from nature, thus, how can any of our actions or creations be 

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            warmup_steps = 5,
            # num_train_epochs = 1, # Set this for 1 full training run.
            max_steps = 60,
            learning_rate = 2e-4,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 1,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 3407,
            output_dir = "outputs",
        ),
)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.458500
2,2.445900
3,2.551600
4,2.640200
5,2.493800
6,2.389000
7,2.191700
8,2.157200
9,2.090700
10,2.006500


In [9]:
from tqdm import tqdm
def infer(dataset, model, instruction):
  results = []
  for line in tqdm(dataset, desc="Inferring on test set: "):
      inputs = tokenizer(
      [
          alpaca_prompt.format(
              instruction, # instruction
              line["input"], # input
              "") # output - leave this blank for generation!
      ], return_tensors = "pt").to("cuda")
      outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
      result = tokenizer.batch_decode(outputs)[0]

      start_marker = "### Response:\n"
      end_marker = "<|end_of_text|>"
      result = result[result.find(start_marker) + len(start_marker):
                      result.find(end_marker)]

      results.append(result)
  return results

test_dataset = dataset["test"]

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
FastLanguageModel.for_inference(model_original)

instruction = test_dataset[0]["instruction"]
original_instruction = instruction

benchmark_results = infer(test_dataset, model_original, original_instruction)
ft_results = infer(test_dataset, model, instruction)

Inferring on test set: 100%|██████████| 200/200 [00:57<00:00,  3.46it/s]


In [10]:
targets = [line["output"][:1] for line in test_dataset]

def accuracy(results, targets):
    results = [result.lower()[:1] for result in results]
    def check_pred(preds):
        n_mismatches = 0
        for i, p in (enumerate(preds)):
            if p not in ['f', 's']:
                if p not in ['m', 'r']:
                    n_mismatches += 1
        if n_mismatches:
            print(f"{n_mismatches}/{len(preds)} "
              f"of the predictions are not in correct format! "
              f"They will not be included in counting accuracy.")
        return n_mismatches
    n_mismatches = check_pred(results)
    
    correct_count = 0
    total = len(results)
    for i in range(total):
      if results[i] == targets[i]:
        correct_count += 1

    return (correct_count - n_mismatches) / (total - n_mismatches)

accu_bm = accuracy(benchmark_results, targets)
accu_ft = accuracy(ft_results, targets)
print(f"The original model accuracy on test set is {accu_bm:.3f}")
print(f"The finetuned model accuracy on test set is {accu_ft:.3f}")

The original model accuracy on test set is 0.540
The finetuned model accuracy on test set is 0.530
